In [5]:
import numpy as np
import pandas as pd
import tweepy
import re
import matplotlib.pyplot as plt
import time

In [6]:
%matplotlib notebook
plt.rcParams['animation.html'] = 'jshtml'

In [7]:
t=tfv.transform(["she is irritating"])
r=model.predict_proba(t)[:,1][0]


In [8]:
def import_tweets(filename, header = None):
    tweet_dataset = pd.read_csv(filename, encoding = "ISO-8859-1", header = header)
    tweet_dataset.columns = ['sentiment','id','date','flag','user','text']
    for i in ['flag','id','user','date']: del tweet_dataset[i] # or tweet_dataset = tweet_dataset.drop(["id","user","date","user"], axis = 1)
    tweet_dataset.sentiment = tweet_dataset.sentiment.replace(4,1)
    return tweet_dataset

def preprocess_tweet(tweet):
    tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet = re.sub('@[^\s]+','AT_USER', tweet)
    #correct all multiple white spaces to a single white space
    tweet = re.sub('[\s]+',' ',tweet)
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    return tweet

#apply the preprocess function for all the tweets in the dataset
tweet_dataset = import_tweets('tweets.csv')
tweet_dataset['text'] = tweet_dataset['text'].apply(preprocess_tweet)
data = np.array(tweet_dataset.text)
label = np.array(tweet_dataset.sentiment)
from sklearn.feature_extraction.text import TfidfVectorizer
tfv=TfidfVectorizer(sublinear_tf=True, stop_words = "english")
features=tfv.fit_transform(data)


from sklearn.linear_model import LogisticRegression
model = LogisticRegression(C=1.)
model.fit(features, label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json
import sqlite3
from unidecode import unidecode
import time



#consumer key, consumer secret, access token, access secret.
ckey = "your consumer_key"
csecret = "your consumer_secret"
atoken = "your access_token"
asecret = "your access_ecret"


conn = sqlite3.connect('twitterbbb.db')
c = conn.cursor()

def create_table():
    try:
        c.execute("CREATE TABLE IF NOT EXISTS sentiment(unix REAL, tweet TEXT, r REAL)")
        c.execute("CREATE INDEX fast_unix ON sentiment(unix)")
        c.execute("CREATE INDEX fast_tweet ON sentiment(tweet)")
        c.execute("CREATE INDEX fast_sentiment ON sentiment(r)")
        conn.commit()
    except Exception as e:
        print(str(e))
create_table()



class listener(StreamListener):

    def on_data(self, data):
        try:
            data = json.loads(data)
            tweet = unidecode(data['text'])
            time_ms = data['timestamp_ms']
            tweet = preprocess_tweet(tweet)
            t = tfv.transform([tweet])
            r = model.predict_proba(t)[:,1][0]
            print(time_ms, tweet, r)
            print("\n\n")
            c.execute("INSERT INTO sentiment (unix, tweet, r) VALUES (?, ?, ?)",
                  (time_ms, tweet, r))
            conn.commit()

        except KeyError as e:
            print(str(e))
        return(True)

    def on_error(self, status):
        print(status)


while True:

    try:
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        twitterStream = Stream(auth, listener())
        twitterStream.filter(track=["modi"] ,languages=["en"])
    except Exception as e:
        print(str(e))
        time.sleep(5)

index fast_unix already exists
1555653956063 RT AT_USER Massive! Modi Govt grants emergency powers to Armed Forces; Army planning to buy 246 Spike Anti-tank Guided Missiles ht... 0.5260634788676917



1555653956345 RT AT_USER As a student of History , I have never heard or read JawaharLalNehru, LalBahadurShastri ManMohanSingh AtalBihariVajpayee ... 0.6160729668742219



1555653956462 RT AT_USER "I'm confident that we will be blessed by the people yet again with a massive mandate, with more seats than before. A strong... 0.8630796055979251



1555653956660 RT AT_USER Baba Ramdev: Muslim&amp;Christian nations don't want PM Modi to win election In other news, Modi ji wins highest civilian... 0.6180443599251285



1555653956975 "Biopic Joke, Make Comedy Film On Him": Urmilac. Madam : Your are from film industry . You out to know Modi is not... URL 0.7673838111275766



1555653958090 RT AT_USER This is funny -- Lalit Chor Modi AT_USER who himself is a fugitive &amp; has fled from India is t

1555653973314 RT AT_USER The boycott the Global Times complained about is not an officially-declared trade war, although Prime Minister Modi expre... 0.7133286157528753



1555653973446 RT AT_USER Modi govt made India's longest Bhupen Hazarika Bridge Asia's 2nd largest Bogibeel Bridge Soon we'll hv Chenab Bridge- Wo... 0.5642550755985187



1555653973814 AT_USER AT_USER Modi takes away crops, Their life Their living Their earnings and what not, Still... URL 0.5372585856541072



1555653973951 Post graduate in commerce &amp; CEO of a 1000 crore Distribution group In Maharashtra 0.7648133986071118



1555653974032 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555653974050 RT AT_USER 50 lakh men from the unregularised sector lose their jobs, because Modi government put up a show of taking on black money.... 0.3105902378176222



1555653974641 RT AT_USER AT_USER AT_USER AT_USER A

1555653994543 RT AT_USER "We have been devastated as we are mired in debt. Crop worth Rs 40,000 has been spoiled." Close to 10 acres of land was us... 0.44747127967778205



1555653994247 N Ram, journalist &amp; controller of The Hindu News Paper, &amp; a certified Anti Modi &amp; Anti Hindu in conversation with D... URL 0.7122400146724267



1555653996237 RT AT_USER Sadhvi Pragya Thakur , A terror accused and BJPs candidate from Bhopal says that 26/11 Martyr Hemant KARKARE was killed by... 0.30670994381921446



1555653996966 RT AT_USER People of Thiruvananthapuram gives our PM AT_USER a welcome that'll give goosebumps even to the Opposition! EveryV... 0.9399782780461645



1555653996987 RT AT_USER Sadhvi Pragya Thakur , A terror accused and BJPs candidate from Bhopal says that 26/11 Martyr Hemant KARKARE was killed by... 0.30670994381921446



1555653997212 RT AT_USER 2014 was a mandate for hope and aspiration. 2019 is about confidence and acceleration. Read my interview to AT_USER 0

1555654012708 'Naamdaar' wants to apply 'Amethi's model of development' in Kerala Watch Modi's latest speeches in 3d view, go to... URL 0.5694403060715079



1555654013917 RT AT_USER Sadhvi Pragya Thakur , A terror accused and BJPs candidate from Bhopal says that 26/11 Martyr Hemant KARKARE was killed by... 0.30670994381921446



1555654014265 EveryVoteForModi Health for all, housing for all, electricity for all - every scheme of Modi government has been a... URL 0.5582836927293622



1555654014339 RT AT_USER Equate this with Modi doing that cringeworthy " Maa bartan dhona paani bharna " dialogue. These people are cheap drama arti... 0.660101603395199



1555654014381 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555654014469 RT AT_USER Context URL 0.9200719067086672



1555654014995 RT AT_USER Bombay blasts : Yakub Memon's scooter used. He attended conspiracy meetings. Maleg

1555654031752 RT AT_USER In Radia tape, one of Mota Bhai's stooge said "Congress toh apni dukaan hai". Despite this proven links of Nehru-Gandhi Pariv... 0.6612834709459822



1555654032016 RT AT_USER Media/liberals try so hard to build this narrative that most Modi bhakts/Right Wingers are these illiterate ghaats from Hindi... 0.736926704899561



1555654032278 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555654032962 RT AT_USER Bombay blasts : Yakub Memon's scooter used. He attended conspiracy meetings. Malegaon blasts : Sadhvi Pragya's motorcycle use... 0.64377954410855



1555654033634 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555654034029 RT AT_USER "I'm confident that we will be blessed by the people yet again with a massive mandate, with more seats than 

...

1555654399446 RT AT_USER Bombay blasts : Yakub Memon's scooter used. He attended conspiracy meetings. Malegaon blasts : Sadhvi Pragya's motorcycle use... 0.64377954410855



1555654399497 RT AT_USER 2014 was a mandate for hope and aspiration, 2019 is about confidence and acceleration: PM AT_USER URL 0.719286372403494



1555654399553 RT AT_USER When they go low, they always find a way to go lower. Atal Bihari Vajpayee's funeral procession viral as PM Modi filing no... 0.26699396471271264



1555654399597 RT AT_USER PM AT_USER tells an election rally, he called Pakistan's nuclear bluff because "India has the mother of all nuclear b... 0.30074314471664276



1555654399444 RT AT_USER One cannot question the AT_USER on flimsy grounds such as Murder. ECI is busy banning movies on Modi AT_USER boo... 0.5648368484744701



1555654399975 RT AT_USER Hi everyone, Thank God I'm safe. I received minor injuries on my back &amp; a stitch on my elbow after I was attacked by TMC go... 0.65156464634534

1555654425010 AT_USER PEOPLE TRUST PM MODI NE MUMKIN KIYA IN 5 YRS MODI NE NYAY KIYA DIRECT TRANSFERS NO AGENTS SPEAK ONLY... URL 0.5186006931951184



1555654425075 AT_USER AT_USER Thats why modi shud have gone full throttle and shud have made ram mandir and removed 370.... URL 0.33437963460405123



1555654425652 AT_USER Modi, a few weeks ago: No Hindu, EVER, has been a terrorist. Modi, a few days ago: A Hindu who is a terro... URL 0.6025258410103029



1555654425902 RT AT_USER Look how Thiruvananthapuram welcomed PM Modi... These visuals will truly give jitters to Shashi Tharoor! EveryVoteForModi... 0.8844547602469018



1555654426546 RT AT_USER Do read - Myths about Nyay busted. Nyay will remonetize what Modi ji demonetized. IndiaMaangeNyay URL 0.7072308276351913



1555654426594 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555654426790 RT AT_USER Ace Painter Mamta didi 

1555654443290 Biopic on PM Modi a joke as he didn`t fulfil promises: Urmila Matondkar URL 0.4938063757838265



1555654443596 RT AT_USER South Indians hate Modi more than North Indians. North Indians hate him because he's Sanghi. South Indians hate him because... 0.2389598934704985



1555654443634 RT AT_USER Ace Painter Mamta didi says there will be non UPA, Non NDA govt in the center, she has tripled farmers income in West Bengal... 0.5256189502833906



1555654443627 RT AT_USER See how Modi declared a War on India's Enviornment. - Twisting Laws to benefit Polluting industries - Erasing Widlife Sa... 0.6414801288716062



1555654443574 Rejection effect is real on ground. INC will find it difficult to capitalize on this.. Many think that majboor sark... URL 0.27643078030508633



1555654443735 RT AT_USER At 3 am, he goes to Supreme Court to get mercy for proven Terrorist Yakub Memon. But he has a problem with Sadhavi's cand... 0.4016001359571587



1555654443990 AT_USER AT_USER Only J

1555654461059 RT AT_USER After near completion of voting in nearly 30% seats one can safely say this is second successive Loksabha general election wh... 0.7098098834758384



1555654461855 RT AT_USER Sadhvi Pragya Thakur , A terror accused and BJPs candidate from Bhopal says that 26/11 Martyr Hemant KARKARE was killed by... 0.30670994381921446



1555654462140 Modi govt 3rd most trusted in the world; Switzerland, Indonesia top list URL 0.7346969807835921



1555654462365 RT AT_USER What is Narendra Modi carrying in his helicopter? Why is he so scared of inspection?" Questions raised on IAS officer M... 0.509531659390805



1555654463268 RT AT_USER Media/liberals try so hard to build this narrative that most Modi bhakts/Right Wingers are these illiterate ghaats from Hindi... 0.736926704899561



1555654463301 RT AT_USER PM AT_USER tells an election rally, he called Pakistan's nuclear bluff because "India has the mother of all nuclear b... 0.30074314471664276



1555654464020 RT AT_USER

1555654485350 RT AT_USER Spoke to AT_USER on Corruption-Free Governance, our prospects in upcoming elections, our vision for the future &amp;... 0.6182491034319721



1555654486407 RT AT_USER Sadhvi Pragya Thakur , A terror accused and BJPs candidate from Bhopal says that 26/11 Martyr Hemant KARKARE was killed by... 0.30670994381921446



1555654486604 RT AT_USER Manufacturing clean chit for Hindu terrorists was the initial step towards getting them into parliament. Sadhvis are Hin... 0.6273347036173602



1555654486634 IndiaBoleModiDobara BharatMangeModiDobara Because India is now safer &amp; better for women. Free LPG connections, sa... URL 0.8616868723960731



1555654486781 AT_USER very sad if Modi gets one more term we will have similar incidents in India too 0.23193329516245093



1555654486911 RT AT_USER Can we blame Manmohan for Kingfisher airlines? No A Journalist blaming Modi over Jet Airways. But He won't tell you that:... 0.5859982672751277



1555654487761 RT AT_USER Bihar